In [1]:
import sys
import os
from modules.CARLA import carla

Appending path to carla module D:\Research\ecemosp\src\modules\CARLA
Appending path to carla module D:\Research\ecemosp\src\modules\CARLA


C:\Users\ignacy\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import joblib
import tensorflow as tf
from carla.recourse_methods import GrowingSpheres, Dice, ActionableRecourse, CEM, CausalRecourse, Face
from carla.models.catalog import MLModelCatalog

In [3]:
def display_results(model, factual, counterfactuals):
    pred = model.predict(factual)
    print('Factual class:', pred.round(3)) 
    print('Counterfactuals class:')
    for pred in model.predict(counterfactuals):
        print(pred.round(3))

In [4]:
from carla.data.catalog import CsvCatalog
import json

with open('../data/adult_constraints.json', 'r') as f:
    constraints = json.load(f)

continuous = constraints['continuous']
categorical = constraints['categorical']
immutable = constraints['immutable']
columns_order = constraints['features_order']

dataset = CsvCatalog(file_path="../data/adult_prep.csv",
                    continuous=continuous,
                    categorical=categorical,
                    immutables=immutable,
                    target='income')
                    
dataset.df.head(2)

,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,1.000000,0.533333,0.0,1.0,0.397959,...,0.0,0.0,0.0,0.0,0.0
1,0.890411,0.533333,0.0,1.0,0.173469,...,1.0,0.0,0.0,0.0,0.0


In [5]:
from models.tfmodel import TFModelAdult
model = TFModelAdult(dataset)

DICE FOR NN

In [17]:
factual = dataset.df_test.iloc[5:6]

hyperparams = {"num": 20}
# load a recourse model and pass black box model
gs = Dice(model, hyperparams)

# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factual)
counterfactuals

,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,0.383562,0.600000,0.2,0.0,0.602041,...,0.0,0.0,1.0,0.0,0.0
1,0.383562,0.600000,0.0,0.0,0.602041,...,0.0,0.0,1.0,0.0,0.0
2,0.383562,0.600000,0.0,0.8,0.602041,...,0.0,0.0,1.0,0.0,0.0
3,0.383562,0.600000,0.5,0.0,0.602041,...,0.0,0.0,1.0,0.0,0.0
4,0.383562,0.600000,0.4,0.0,0.602041,...,0.0,0.0,1.0,0.0,0.0
5,0.383562,0.600000,0.0,1.0,0.602041,...,1.0,0.0,1.0,0.0,0.0
6,0.383562,0.600000,0.0,1.1,0.602041,...,0.0,0.0,1.0,0.0,0.0
7,0.383562,0.600000,0.9,0.0,0.602041,...,0.0,0.0,1.0,0.0,0.0
8,0.383562,0.600000,1.0,0.0,0.602041,...,0.0,0.0,1.0,0.0,0.0
9,0.383562,0.600000,0.0,0.6,0.602041,...,0.0,0.0,1.0,0.0,0.0


In [18]:
display_results(model, factual, counterfactuals)

Factual class: [[0.615 0.385]]
Counterfactuals class:
[0.003 0.997]
[0.476 0.524]
[0.013 0.987]
[0. 1.]
[0. 1.]
[0.036 0.964]
[0.009 0.991]
[0. 1.]
[0. 1.]
[0.36 0.64]
[0. 1.]
[0. 1.]
[0.489 0.511]
[0.392 0.608]
[0. 1.]
[0.029 0.971]
[0.13 0.87]
[0.089 0.911]
[0.307 0.693]
[0.319 0.681]


CAUSAL RECOURSE

In [16]:
# from carla.recourse_methods.catalog.causal_recourse import samplers, constraints
# hypers = {
#     'optimization_approach': "brute_force",
#     'num_samples': 10,
#     'scm': None,
#     "constraint_handle": constraints.point_constraint,
#     "sampler_handle": samplers.sample_true_m0,
# }


# gs = CausalRecourse(model, hypers)
# # generate counterfactual examples
# counterfactuals = gs.get_counterfactuals(dataset.df[:1])
# counterfactuals

GROWING SPHERES

In [21]:
gs = GrowingSpheres(model)
# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factual)
counterfactuals

,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
9279,0.383562,0.819483,1.315817,0.100825,0.977128,...,1.0,0.0,1.0,1.0,0.0


In [9]:
display_results(model, factual, counterfactuals)

Factual class: [[0.615 0.385]]
Counterfactuals class:
[0. 1.]


ACTIONABLE RECOURSE (AR)

In [28]:
hyperparams = {
    'fs_size': 100
}

ar = ActionableRecourse(model, hyperparams)
counterfactuals = ar.get_counterfactuals(factual)
counterfactuals

[INFO] Start generating LIME coefficients [model.py get_counterfactuals]
[INFO] Finished generating LIME coefficients [model.py get_counterfactuals]
obtained 100 items in 1.2 seconds


,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,0.383562,0.6,0.0,0.0,0.602041,...,0.0,0.0,1.0,0.0,0.0


In [26]:
display_results(model, factual, counterfactuals)

Factual class: [[0.615 0.385]]
Counterfactuals class:
[0.39 0.61]


CONTRASTIVE EXPLANATIONS METHOD (CEM)

In [12]:
hyperparams = {
        "data_name": 'test1',
        "batch_size": 1,
        "kappa": 0.1,
        "init_learning_rate": 0.01,
        "binary_search_steps": 9,
        "max_iterations": 100,
        "initial_const": 10,
        "beta": 0.9,
        "gamma": 0.0,
        "mode": "PN",
        "num_classes": 2,
        "ae_params": {"hidden_layer": [20, 10, 7], "train_ae": True, "epochs": 5},
}


sess = None

cm = CEM(sess = sess, mlmodel=model, hyperparams=hyperparams)

counterfactuals = cm.get_counterfactuals(factual)
counterfactuals

Train on 22792 samples, validate on 9769 samples
Epoch 1/5
22792/22792 [==============================] - 1s 28us/sample - loss: 0.0646 - val_loss: 0.0311
Epoch 2/5
 3136/22792 [===>..........................] - ETA: 0s - loss: 0.0306

c:\Users\ignacy\.conda\envs\carla_test\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


22792/22792 [==============================] - 0s 18us/sample - loss: 0.0259 - val_loss: 0.0231
Epoch 3/5
22792/22792 [==============================] - 0s 17us/sample - loss: 0.0216 - val_loss: 0.0205
Epoch 4/5
22792/22792 [==============================] - 0s 18us/sample - loss: 0.0195 - val_loss: 0.0190
Epoch 5/5
22792/22792 [==============================] - 0s 18us/sample - loss: 0.0184 - val_loss: 0.0189


AttributeError: module 'tensorflow' has no attribute 'placeholder'

Feasible and Actionable Counterfactual Explanations (FACE)

In [30]:
hyperparams = {
    'mode': 'knn',
    #'fraction':,
}
fc = Face(model, hyperparams)

counterfactuals = fc.get_counterfactuals(factual)
counterfactuals


,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
9279,0.479452,0.8,0.0,0.0,0.5,...,0.0,0.0,1.0,0.0,0.0


In [31]:
display_results(model, factual, counterfactuals)

Factual class: [[0.615 0.385]]
Counterfactuals class:
[0.471 0.529]
